Here we will go through the crucial steps from the main.py file (steps, which are related to checks are omitted). First, some modules are loaded

In [16]:
import Cowtan
import sys

In the input file (here "file.inp") the commuting Pauli strings with coefficients, which will form the Pauli gadgets, are entered line by line

In [26]:
fl_inp = open("file.inp","r")

ps_arr = []
coef_arr = []
for ln in fl_inp.readlines():
    p, c = ln.split()
    ps_arr.append(p)
    coef_arr.append(float(c))
    print(p,c)

YXXX -0.125
XYXX -0.125
XXYX 0.125
YYYX -0.125
XXXY 0.125
YYXY -0.125
YXYY 0.125
XYYY 0.125


This input is equivalent to 

exp{i (XXXY + XXYX - XYXX - YXXX - YYYX - YYXY + YXYY + XYYY) / 8}

This is the double excitation operator in the Jordan-Wigner mapping

First the circuit is created

In [27]:
Nq = len(ps_arr[0])
print("Number of qubits = ", Nq)

# Create the circuit
zx_circ = Cowtan.zx_circ_cls(Nq)
for i,ps in enumerate(ps_arr):
    zx_circ.add_pauli_gadget(ps, coef=coef_arr[i])

print("\n",zx_circ)

Number of qubits =  4

 -0.125*YXXX_3210 -0.125*XYXX_3210 0.125*XXYX_3210 -0.125*YYYX_3210 0.125*XXXY_3210 -0.125*YYXY_3210 0.125*YXYY_3210 0.125*XYYY_3210 


Here ```coef*P_alpha``` stands for the Pauli gadget ```e^{i*coef*P}```, where ```alpha``` indicates on which qubits particular Pauli matrix acts on.

Now we launch the Cowtan-Simmons-Duncan algorithm

In [28]:
zx_circ.cowtan()
print(zx_circ)

p_3 H_2 H_1 H_0 CX_01 CX_12 CX_23 H_0 H_1 p_2 -0.125*Z_3 0.125*Z_2 -0.125*ZZ_21 0.125*ZZ_31 -0.125*ZZZ_210 0.125*ZZZ_310 -0.125*ZZ_30 0.125*ZZ_20 m_2 H_1 H_0 CX_23 CX_12 CX_01 m_3 H_2 H_1 H_0 


Here:
- ```H_i``` stands for the Hadamard gate acting on ```i``` qubit
- ```CX_ij``` stands for the CNOT gate with the control and target qubits being ```i``` and ```j```, respectively
- ```p_i``` and ```m_i``` stands for the ```R_x(pi/2)``` and ```R_x(-pi/2)``` gates acting on ```i``` qubit

It is seen that all Pauli gadgets are converted to phase gadgets ```exp(i*theta*P)``` with ```P``` consisting only out of ```Z``` and ```I``` matrices.

Now we apply the Vandaele-Martiel-Brugiere and Patel-Markov-Hayes algorithms

In [29]:
zx_circ.vandaele_patel()
print(zx_circ)

p_3 H_2 H_1 H_0 CX_01 CX_12 CX_23 H_0 H_1 p_2 -0.125*Z_3 0.125*Z_2 CX_02 0.125*Z_2 CX_12 -0.125*Z_2 CX_02 -0.125*Z_2 CX_30 -0.125*Z_0 CX_10 0.125*Z_0 CX_31 0.125*Z_1 CX_12 CX_32 CX_10 CX_31 m_2 H_1 H_0 CX_23 CX_12 CX_01 m_3 H_2 H_1 H_0 


We can also see the number of single-qubit and CNOT gates used in this circuit

In [30]:
nsqC, ncx = zx_circ.stats()
print( "Single Qubit Gates = ", nsqC, "CNOTs = ", ncx )

Single Qubit Gates =  22 CNOTs =  16


Let us note that direct (greedy) convertation of the initial Phase gadgets to the  circuit one gets 40 CNOT gates and 56 single-qubit gates.

It is also possible to represent the circuit as a set of comands for the ```quantikz```. 

In [32]:
zx_circ.qasm()

\qw & \qw & \qw & \gate{H} & \ctrl{1} & \qw & \qw & \gate{H} & \qw & \qw & \qw & \qw & \ctrl{2} & \qw & \qw & \qw & \ctrl{2} & \qw & \targ{} & \gate{R_z(-0.125)} & \targ{} & \gate{R_z(0.125)} & \qw & \qw & \qw & \qw & \targ{} & \qw & \qw & \qw & \gate{H} & \qw & \qw & \ctrl{1} & \qw & \qw & \qw & \gate{H} & 
\\
\qw & \qw & \gate{H} & \qw & \targ{} & \ctrl{1} & \qw & \qw & \gate{H} & \qw & \qw & \qw & \qw & \qw & \ctrl{1} & \qw & \qw & \qw & \qw & \qw & \ctrl{-1} & \qw & \targ{} & \gate{R_z(0.125)} & \ctrl{1} & \qw & \ctrl{-1} & \targ{} & \qw & \gate{H} & \qw & \qw & \ctrl{1} & \targ{} & \qw & \qw & \gate{H} & \qw & 
\\
\qw & \gate{H} & \qw & \qw & \qw & \targ{} & \ctrl{1} & \qw & \qw & \gate{V} & \qw & \gate{R_z(0.125)} & \targ{} & \gate{R_z(0.125)} & \targ{} & \gate{R_z(-0.125)} & \targ{} & \gate{R_z(-0.125)} & \qw & \qw & \qw & \qw & \qw & \qw & \targ{} & \targ{} & \qw & \qw & \gate{V^\dagger} & \qw & \qw & \ctrl{1} & \targ{} & \qw & \qw & \gate{H} & \qw & \qw & 
\\
\gate{V} & \qw & 

This code will produce the following circuit
![title](graph/double_complete.png)